In [ ]:
import pandas as pd
from datetime import datetime
pd.options.display.max_columns = 100

In [ ]:
%%bigquery bq_df
SELECT
  chnl_szr_dt AS CALL_DATE,
  ban AS BAN,
  subscr_num AS SUBSCRIBER_NUM,
  call_to_tn_txt AS CALL_TO_TN,
  call_to_city_desc_str AS CALL_TO_CITY,
FROM `cio-datahub-enterprise-pr-183a.ent_usage_rated.bq_wls_ptpd_unbilld_dly_call_usg_evnt_dtl` 
WHERE chnl_szr_dt >= '2022-06-28'
AND (
       UPPER(call_to_city_desc_str) LIKE 'UKRAINE%'
    OR UPPER(call_to_city_desc_str) LIKE 'BELARUS%'
    OR UPPER(call_to_city_desc_str) LIKE 'HUNGARY%'
    OR UPPER(call_to_city_desc_str) LIKE 'MOLDOVA%'
    OR UPPER(call_to_city_desc_str) LIKE 'POLAND%'
    OR UPPER(call_to_city_desc_str) LIKE 'ROMANIA%'
    OR UPPER(call_to_city_desc_str) LIKE 'SLOVAK%'
    OR UPPER(call_to_city_desc_str) LIKE 'RUSSIA%'
)
ORDER BY chnl_szr_dt

In [ ]:
hive_df = pd.read_csv('data/july_25_2022_wireless_calls.csv')

In [ ]:
print(hive_df.call_date.min(), hive_df.call_date.max())

In [ ]:
print(bq_df.CALL_DATE.min(), bq_df.CALL_DATE.max())

In [ ]:
bq_df['date']=bq_df.CALL_DATE.apply(lambda x: x.date())
hive_df['date']=hive_df.call_date.apply(lambda x: x.split(' ')[0])

In [ ]:
hive_df.groupby('date').count()[['ban']].transpose()

In [ ]:
bq_df.groupby('date').count()[['BAN']].transpose()

In [ ]:
%%bigquery wireless_df_new
SELECT
  chnl_szr_dt AS CALL_DATE,
  ban AS BAN,
  subscr_num AS SUBSCRIBER_NUM,
  call_to_tn_txt AS CALL_TO_TN,
  call_to_city_desc_str AS CALL_TO_CITY,
FROM `cio-datahub-enterprise-pr-183a.ent_usage_rated.bq_wls_ptpd_unbilld_dly_call_usg_evnt_dtl` 
WHERE chnl_szr_dt >= '2022-01-28'
AND (
       UPPER(call_to_city_desc_str) LIKE 'UKRAINE%'
    OR UPPER(call_to_city_desc_str) LIKE 'BELARUS%'
    OR UPPER(call_to_city_desc_str) LIKE 'HUNGARY%'
    OR UPPER(call_to_city_desc_str) LIKE 'MOLDOVA%'
    OR UPPER(call_to_city_desc_str) LIKE 'POLAND%'
    OR UPPER(call_to_city_desc_str) LIKE 'ROMANIA%'
    OR UPPER(call_to_city_desc_str) LIKE 'SLOVAK%'
    OR UPPER(call_to_city_desc_str) LIKE 'RUSSIA%'
)
ORDER BY chnl_szr_dt

In [ ]:
wireless_df_new

In [ ]:
wireless_df_new=wireless_df_new.drop_duplicates(subset=['SUBSCRIBER_NUM', 'CALL_DATE']).reset_index(drop=True)

In [ ]:
wireless_df_new.shape

In [ ]:
wireless_df_new['CALL_DATE']=wireless_df_new.CALL_DATE.apply(lambda x: x.date())
wireless_df_new['CALL_DATE']=pd.to_datetime(wireless_df_new['CALL_DATE'])

In [ ]:
wireless_df=wireless_df_new.copy()
wireless_df['call_to_city_list'] = wireless_df.CALL_TO_CITY.str.split(' ')
# wireless_df['call_date']=wireless_df['call_date'].apply(lambda x: str(x).split(' ')[0])
# wireless_df['call_date']=pd.to_datetime(wireless_df['call_date'])
wireless_df_2=wireless_df.explode('call_to_city_list')
countries_list = ['UKRAINE', 'BELARUS', 'HUNGARY', 'MOLDOVA', 'POLAND', 'ROMANIA', 'SLOVAKIA', 'RUSSIA']
wireless_df_3=wireless_df_2[wireless_df_2.call_to_city_list.isin(countries_list)].reset_index(drop=True)

In [ ]:
wireless_df_3

In [ ]:
wireless_df_3.loc[(wireless_df_3.CALL_DATE>'2022-07-01') & (wireless_df_3.CALL_TO_CITY=='UKRAINE')].SUBSCRIBER_NUM.nunique()